# Import

In [1]:
import pickle

# math and dataframes
import pandas as pd
import numpy as np

# statistics
from sklearn.metrics import r2_score
from scipy.stats import friedmanchisquare, wilcoxon
from sklearn.decomposition import PCA

# Pipeline and Evaluation
from sklearn.metrics import classification_report, confusion_matrix

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set_theme()

In [2]:
df_predictions = pd.read_pickle('df_predictions.pickle')

In [3]:
# columns for datasets

y_column = 'in_B100'
X_columns = [
    'mode', 'acousticness', 'danceability', 'duration_ms', 'energy',
    'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'
]
genre_columns = [
    'is_Adult_Standard', 'is_Rock', 'is_R&B', 'is_Country', 'is_Pop',
    'is_Rap', 'is_Alternative', 'is_EDM', 'is_Metal'
]
cluster_columns = ['cluster', 'cluster2']
other_columns = ['key', 'time_signature', 'genre', 'release_date']
prediction_columns = ['y_lr', 'y_dt', 'y_knn', 'y_rf', 'y_ab', 'y_lr_tuned', 'y_dt_tuned', 'y_cl_1', 'y_cl_2', 'y_genres']

# Explore Predictions

In [4]:
df_predictions[prediction_columns].describe().T

,count,unique,top,freq
y_lr,8827719,2,False,5012328
y_dt,8827719,2,False,5759549
y_knn,8827719,2,False,5389246
y_rf,8827719,2,False,5972306
y_ab,8827719,2,False,5368640
y_lr_tuned,8827719,2,False,5015482
y_dt_tuned,8827719,2,False,5730894
y_cl_1,8827719,2,False,5524314
y_cl_2,8827719,2,False,5445396
y_genres,2808901,2,False,1693361


In [5]:
df_predictions[~df_predictions.y_genres.isna()][prediction_columns].describe().T

,count,unique,top,freq
y_lr,2808901,2,False,1632876
y_dt,2808901,2,False,1856456
y_knn,2808901,2,False,1747625
y_rf,2808901,2,False,1934264
y_ab,2808901,2,False,1746821
y_lr_tuned,2808901,2,False,1633972
y_dt_tuned,2808901,2,False,1855688
y_cl_1,2808901,2,False,1792586
y_cl_2,2808901,2,False,1769889
y_genres,2808901,2,False,1693361


In [ ]:
# Friedman Test (Wilcoxn if required)


In [ ]:
# Histograms 
## NEED TO COMBINE WITH id to compare AF
